In [ ]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.schema import Document
import json
import os
from dotenv import load_dotenv

load_dotenv()
key = os.getenv("OPENAI_API_KEY")


# 1. 데이터 불러오기
try:
    with open("./vector_db_final.json", "r", encoding="utf-8") as f:
        raw_data = json.load(f)
except FileNotFoundError:
    print("파일 없음")
    exit()

# 2. Document 구조로 변환
docs = [
    Document(
        page_content=item["text"],
        metadata=item["metadata"]
    )
    for item in raw_data
]

# 3. OpenAI 임베딩 초기화
embed_model = OpenAIEmbeddings(

    model="text-embedding-3-small",
    openai_api_key=key
    )  # 또는 "text-embedding-3-large"

# 4. ChromaDB에 저장
vectordb = Chroma.from_documents(
    documents=docs,
    embedding=embed_model,
    persist_directory="./chroma_childcare"
)

# 5. 저장
vectordb.persist()


In [32]:
# from langchain.chains import RetrievalQA
# from langchain.prompts import PromptTemplate
# from langchain.chat_models import ChatOpenAI  # 또는 OpenAI()

# # 💬 사용자 질의에 사용할 프롬프트 템플릿
# prompt = PromptTemplate(
#     input_variables=["context", "question"],
#     template="""
# 당신은 영유아 부모를 위한 육아 상담 챗봇입니다.
# 아래 문맥(Context)을 기반으로 질문에 정확하고 친절하게 답해주세요.
# 만약 

# [문맥]
# {context}

# [질문]
# {question}

# [답변]
# """
# )

# # 🔍 벡터 DB를 기반으로 retriever 생성
# retriever = vectordb.as_retriever(search_kwargs={"k": 3})  # 상위 3개 문서 검색

# # 💡 ChatGPT 또는 GPT-4o 모델 활용
# qa_chain = RetrievalQA.from_chain_type(
#     llm=ChatOpenAI(model_name="gpt-4o", temperature=0.5, openai_api_key=key),
#     retriever=retriever,
#     chain_type="stuff",  # stuff는 단순한 문서 삽입 방식
#     chain_type_kwargs={"prompt": prompt}
# )

# query = " 아이가 밤에 자꾸 깨는데 어떻게 재워야 하나요?"
# # docs = vectordb.similarity_search(query, k=5)
# response = qa_chain.run(query)

# print("📌 질문:", query)
# print("🧠 답변:", response)

# # for doc in docs:
# #     print(f"[{doc.metadata['category_name']} / {doc.metadata['page_name']}]")
# #     print(doc.page_content)
# #     print("=" * 50)

In [40]:
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
import re

def detect_missing_info(question: str) -> str | None:
    if not re.search(r'\b(\d{1,2}\s*개월|[0-9]{1,2}살|신생아|출생|돌|만\s*\d{1,2})\b', question):
        return "아기의 개월 수를 알려주시면 보다 정확한 답변을 드릴 수 있어요 😊"
    return None


# 대화 히스토리 저장 메모리
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# 프롬프트 템플릿
prompt_template = PromptTemplate(
    input_variables=["context", "question", "chat_history"],
    template="""
당신은 영유아 부모의 육아 질문에 친절하게 답하는 상담 챗봇입니다.
이전 대화 내용: {chat_history}

[문맥 정보]
{context}

[질문]
{question}

[답변]
"""
)

# Vector DB 기반 retriever
retriever = vectordb.as_retriever(search_kwargs={"k": 3})

# Conversational Retrieval QA 체인
qa = ConversationalRetrievalChain.from_llm(
    llm=ChatOpenAI(model_name="gpt-4o", temperature=0.3, openai_api_key=key),
    retriever=retriever,
    memory=memory,
    combine_docs_chain_kwargs={"prompt": prompt_template}
)

# 사용자 질문
user_question = input("❓안녕하세요. 궁금한게 있으시면 말씀해주세요")
print(user_question)

# 1. 개월 수 빠졌는지 확인
followup_msg = detect_missing_info(user_question)
if followup_msg:
    print("🤖", followup_msg)
    # 사용자 응답 예시
    user_reply = input()
    print(user_reply)
    final_input = user_question + " (아이 개월 수: " + user_reply + ")"
else:
    final_input = user_question

# 2. 최종 질의 처리
response = qa.run(final_input)
print("🧠 답변:", response)


아이가 열이 많이 나요
🤖 아기의 개월 수를 알려주시면 보다 정확한 답변을 드릴 수 있어요 😊
6개월이요
🧠 답변: 아이가 열이 나는 상황은 부모에게 걱정스러운 일이 될 수 있습니다. 6개월 아기의 경우, 열이 나는 원인은 다양할 수 있으며, 감기, 바이러스 감염, 예방 접종 후 반응 등이 일반적인 원인입니다. 다음은 아이가 열이 날 때 취할 수 있는 몇 가지 조치입니다:

1. **체온 측정**: 아기의 체온을 정확하게 측정하여 열이 얼마나 나는지 확인하세요. 38도 이상이면 열이 있는 것으로 간주됩니다.

2. **수분 공급**: 열이 날 때는 탈수 위험이 있으므로 아기에게 충분한 수분을 공급하세요. 모유 수유 중이라면 자주 수유하고, 분유를 먹는 아기라면 물을 조금씩 자주 주는 것이 좋습니다.

3. **옷차림 조절**: 아기가 너무 덥지 않도록 가벼운 옷을 입히고, 방 안의 온도를 적절하게 유지하세요.

4. **해열제 사용**: 필요하다면 소아과 의사와 상담 후 아기에게 적절한 해열제를 사용할 수 있습니다. 아세트아미노펜(타이레놀)이나 이부프로펜(어린이용 부루펜)이 일반적으로 사용됩니다. 하지만, 의사의 지시 없이 약을 사용하지 않도록 주의하세요.

5. **의사 상담**: 열이 24시간 이상 지속되거나, 아기가 매우 보채거나, 식욕이 없고, 기운이 없어 보이는 경우 즉시 소아과 의사와 상담하세요.

아기의 상태를 주의 깊게 관찰하고, 필요시에는 전문가의 도움을 받는 것이 중요합니다. 아기의 빠른 회복을 바랍니다.
